# MASK RCNN - Evaluate

In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
sys.path.append('.')
sys.path.append('..')
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import lib.mrcnn.model as modellib
from lib.mrcnn import visualize, utils
import lib.mrcnn.utils as mrcnn_utils
from common.utils import get_data_dir, get_model_dir, gpu_ram_growth_config
import logging
from common.log import init_logging
from service_modelTraining.maskrcnn_training import get_dataset, InferenceConfig
#----------
init_logging('maskrcnn_evaluate')
MODEL_DIR = get_model_dir()
gpu_ram_growth_config(0)

MODEL_VER = 'maskrcnn_20220901_best.h5'
model_path = os.path.join(MODEL_DIR, MODEL_VER)

## Load data & model

In [ ]:
data_bunch = get_dataset()

In [ ]:


inference_config = InferenceConfig()
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)


## Inference Demo

In [ ]:
#Get Ground Truth
image_id = 7
original_image, _, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(data_bunch.val, inference_config, image_id)

#Inference
detect_ret = model.detect([original_image], verbose=0)

plt.figure(figsize=(15,10))
ax = plt.subplot(121)
r = detect_ret[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            data_bunch.val.class_names, r['scores'], figsize=(15, 15), ax=ax, title='Inference')
ax = plt.subplot(122)
visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            data_bunch.val.class_names, figsize=(15, 15), ax=ax, title='Ground Truth')


## FPS Test

In [ ]:
fps_list = []
for image_id in tqdm(data_bunch.val.image_ids):
    img = data_bunch.val.load_image(image_id)
    fstart = time.time()
    results = model.detect([img], verbose=0)
    end = time.time()
    fps = np.round(1 / (time.time() - fstart),2)
    fps_list.append(fps)
fps_mean = np.mean(fps_list)
print(f'fps: {fps_mean:.2f}')    

## 計算mAP

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
average_precision_list = []
for image_id in tqdm(data_bunch.val.image_ids):
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(data_bunch.val, inference_config,
                               image_id)
    detect_ret = model.detect([image], verbose=0)
    r = detect_ret[0]
    # Compute AP
    ap, precisions, recalls, overlaps =\
        mrcnn_utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    average_precision_list.append(ap)
mAP = np.nanmean(average_precision_list)
print(f'mAP: {mAP:.2f}', )
display(pd.DataFrame({'image_id':data_bunch.val.image_ids,'ap':average_precision_list}))
